Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [8]:
#由于python和tensorflow版本的更新，不使用上面的引用
import numpy as np
import tensorflow.compat.v1 as tf
import pickle

In [9]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [10]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [11]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    #除四本质上是除2再除2，另外我觉得这里应该是向上取整。
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')#zero padding
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.847403
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.819741
Minibatch accuracy: 18.8%
Validation accuracy: 46.7%
Minibatch loss at step 100: 1.051347
Minibatch accuracy: 56.2%
Validation accuracy: 57.5%
Minibatch loss at step 150: 0.498684
Minibatch accuracy: 87.5%
Validation accuracy: 73.8%
Minibatch loss at step 200: 0.725867
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 250: 1.297786
Minibatch accuracy: 56.2%
Validation accuracy: 76.6%
Minibatch loss at step 300: 0.326843
Minibatch accuracy: 93.8%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.491502
Minibatch accuracy: 93.8%
Validation accuracy: 77.2%
Minibatch loss at step 400: 0.336185
Minibatch accuracy: 93.8%
Validation accuracy: 80.1%
Minibatch loss at step 450: 1.011736
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 500: 0.891347
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

- 卷积-池化-卷积-池化，stride都是2

In [16]:
import math
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [math.ceil(image_size / 16 )* math.ceil(image_size / 16 )* depth, num_hidden], stddev=0.1))
#   layer3_weights = tf.Variable(tf.truncated_normal(
#       [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    #除四本质上是除2再除2，另外我觉得这里应该是向上取整。
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')#zero padding
    hidden = tf.nn.relu(conv + layer1_biases)
    pooling = tf.nn.max_pool(hidden,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
    conv = tf.nn.conv2d(pooling, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pooling = tf.nn.max_pool(hidden,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
    shape = pooling.get_shape().as_list()
    reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

In [17]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.630159
Minibatch accuracy: 12.5%
Validation accuracy: 10.1%
Minibatch loss at step 1000: 0.384666
Minibatch accuracy: 93.8%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 0.171208
Minibatch accuracy: 93.8%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 0.823041
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%
Minibatch loss at step 4000: 0.534170
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 5000: 1.230368
Minibatch accuracy: 81.2%
Validation accuracy: 86.2%
Minibatch loss at step 6000: 0.223081
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 7000: 0.679123
Minibatch accuracy: 81.2%
Validation accuracy: 86.9%
Minibatch loss at step 8000: 0.581371
Minibatch accuracy: 81.2%
Validation accuracy: 87.2%
Minibatch loss at step 9000: 0.406948
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 10000: 0.104929
Minibatch accuracy: 100.0%
Validation accu

- 卷积-池化-卷积-池化，卷积stride是1，池化是2

In [18]:
import math
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
#   layer3_weights = tf.Variable(tf.truncated_normal(
#       [math.ceil(image_size / 16 )* math.ceil(image_size / 16 )* depth, num_hidden], stddev=0.1))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    #除四本质上是除2再除2，另外我觉得这里应该是向上取整。
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')#zero padding
    hidden = tf.nn.relu(conv + layer1_biases)
    pooling = tf.nn.max_pool(hidden,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
    conv = tf.nn.conv2d(pooling, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pooling = tf.nn.max_pool(hidden,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
    shape = pooling.get_shape().as_list()
    reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

In [19]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.439188
Minibatch accuracy: 6.2%
Validation accuracy: 13.0%
Minibatch loss at step 1000: 0.158195
Minibatch accuracy: 93.8%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 0.071068
Minibatch accuracy: 100.0%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 0.744668
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Minibatch loss at step 4000: 0.442715
Minibatch accuracy: 81.2%
Validation accuracy: 87.7%
Minibatch loss at step 5000: 0.947813
Minibatch accuracy: 81.2%
Validation accuracy: 87.5%
Minibatch loss at step 6000: 0.054827
Minibatch accuracy: 100.0%
Validation accuracy: 87.7%
Minibatch loss at step 7000: 0.602835
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 8000: 0.514313
Minibatch accuracy: 81.2%
Validation accuracy: 88.1%
Minibatch loss at step 9000: 0.392903
Minibatch accuracy: 93.8%
Validation accuracy: 89.2%
Minibatch loss at step 10000: 0.173313
Minibatch accuracy: 93.8%
Validation accu

后者效果比前者要好

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

仿照LeNet5,构建神经网络，使用dropout和 learning rate decay.

In [19]:
import math
batch_size = 16
patch_size = 5
another_patch_size = 4
depth1 = 6
depth2 = 16
num_hidden1 = 120
num_hidden2 = 84

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth1]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))

    layer3_weights = tf.Variable(tf.truncated_normal([another_patch_size,another_patch_size, depth2,num_hidden1], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden1, num_hidden2],stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden2, num_labels],stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels])) 
  
  # Model.
    def model(data):
                                                      
    #下面为卷积层C1，结果为length*24*24*6
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')#No padding
        hidden = tf.nn.relu(conv + layer1_biases)
    #下面为池化层S2，结果为length*12*12*6
        pooling = tf.nn.avg_pool(hidden,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'VALID')
    #下面为卷积层C3，结果为length*8*8*16
        conv = tf.nn.conv2d(pooling, layer2_weights, [1, 1, 1, 1], padding='VALID')
        hidden = tf.nn.relu(conv + layer2_biases)
    #下面为池化层S2，结果为length*4*4*16
        pooling = tf.nn.max_pool(hidden,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'VALID')
    #下面为卷积层C5，结果为length*1*1*120
        conv = tf.nn.conv2d(pooling, layer3_weights, [1, 1, 1, 1], padding='VALID')
        hidden = tf.nn.relu(conv + layer3_biases)
    #处理一下方便矩阵运算，结果为length*120                                               
        shape = hidden.get_shape().as_list()                                               
        hidden =  tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
     #dropout
        drop = tf.nn.dropout(hidden,rate = 0.5)                                                
    #下面为全连接层6，结果为length*84
        layer1 = tf.nn.relu(tf.matmul(drop, layer4_weights) + layer4_biases)
      #dropout  
        drop = tf.nn.dropout(layer1,rate = 0.5)                                               
      #下面为全连接层7，结果为length*10                                            
        return (tf.matmul(drop, layer5_weights) + layer5_biases)                                                     
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  
  #  learning rate decay
    global_step = tf.Variable(0,trainable = False)  
    learning_rate = tf.train.exponential_decay(0.05,global_step,1000,0.8,staircase = True)                                                
  # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))    

In [20]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.182978
Minibatch accuracy: 12.5%
Validation accuracy: 10.6%
Minibatch loss at step 1000: 0.656524
Minibatch accuracy: 81.2%
Validation accuracy: 71.2%
Minibatch loss at step 2000: 0.314246
Minibatch accuracy: 87.5%
Validation accuracy: 75.8%
Minibatch loss at step 3000: 1.082729
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 4000: 0.490774
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 5000: 1.487237
Minibatch accuracy: 75.0%
Validation accuracy: 81.3%
Minibatch loss at step 6000: 0.345160
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
Minibatch loss at step 7000: 0.978307
Minibatch accuracy: 68.8%
Validation accuracy: 81.6%
Minibatch loss at step 8000: 0.515905
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
Minibatch loss at step 9000: 0.633451
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
Minibatch loss at step 10000: 0.335700
Minibatch accuracy: 87.5%
Validation accur

调了好几次参数，正确率还是上不去，还不如一开始的卷积加池化。感觉处在欠拟合状态，应该是由于参数太多导致。